# Google Earth Engine API Example

### December 29, 2022

This notebook provides a brief introduction to how to use the Google Earth Engine API to select a few [HUC-4](https://nas.er.usgs.gov/hucs.aspx) level basins (subregions in USGS HUC terminology), combine those basins that are in the same region, use those features to filter hydrography features and clip digital elevation data, and export that data to a Google Drive folder. You would do this, for instance, when you want to use the output data to make a nice map for a report, proposal, or other document. 

This notebook depends on the `ee` and `geemap` libraries, both of which I installed using the `conda install -c conda-forge`. 

__Side note for Mac M1 users:__ I am doing this on a fancy, new 2022 MacBook Pro with the M1 chip. `geemap` *per se* is architecture agnostic, however, some of its dependencies __are not__. Hence, you can install `geemap` that works with the M1 chip architecture, but because not all of the dependencies have recent versions that are optimized for M1 you will likely get an ancient version of `geemap` (like v0.6.x). This will throw some errors in the Earth Engine API that are not easily fixable (ask me how I know). To get around this, I've used Apple's Rosetta emulator to create an `x86_64` Miniconda instance which will be useful as long as there is a version lag in the availability of packages for the `arm64` architecture. This worked like a charm and if you need to do it, I'd highly suggest following the workflow from [this Medium article on the Towards Data Science channel](https://towardsdatascience.com/how-to-install-miniconda-x86-64-apple-m1-side-by-side-on-mac-book-m1-a476936bfaf0).

## 1. Import Libraries and Setup

In [1]:
import subprocess
import ee
import geemap as emap
import numpy as np
import matplotlib.pyplot as plt

# HUC IDs
CU_hucs = ['1704','1705','1706'] # Snake River
CO_hucs = ['1404'] # Green River tributary of the Colorado
MO_hucs = ['1002','1007','1008'] # Yellowstone and Upper Missouri Rivers

# My Google Drive Folder to save the output data too
GDriveSaveFolder = '_C5_Proposal_GEE_Data'


## 2. Authenticate with the Google Earth Engine API

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AWgavdfxWE2o46oxogWzMzhdrGp6Hzq4mXjzNGfdjdK7Pi0KtQ7Ek8B-g2I



Successfully saved authorization token.


## 3. Get Feature Collections from Google Earth Engine 

In [3]:
# 1. Basin boundary dataset:
# USGS HUC-04 subregion collection
huc04 = ee.FeatureCollection('USGS/WBD/2017/HUC04')

# Filter huc04 to get the hucs we need for each subregion
CU_river_huc04 = huc04.filter(ee.Filter.inList('huc4', ee.List(CU_hucs))).union()
CO_river_huc04 = huc04.filter(ee.Filter.inList('huc4', ee.List(CO_hucs))).union()
MO_river_huc04 = huc04.filter(ee.Filter.inList('huc4', ee.List(MO_hucs))).union()

# 2. Basin hydrography dataset:
# WWF HydroSHEDS dataset of free flowing rivers
streams = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers')

# Set up visualization parameters for the HydroSHEDS data
visParams = {
    'lineWidth': 2,
    'color': {
        'property': 'RIV_ORD',
        'mode': 'linear',
        'palette': ['08519c', '3182bd', '6baed6', 'bdd7e7', 'eff3ff'],
        'min': 1,
        'max': 10
    }
}

# Get the HydroSHEDS hydrography for each subregion only within the selected boundaries
CU_river_streams = streams.filterBounds(CU_river_huc04)
CO_river_streams = streams.filterBounds(CO_river_huc04)
MO_river_streams = streams.filterBounds(MO_river_huc04)

## 3. Make an interactive map to ensure that we're getting what we want
Map = emap.Map(center=(45.2607563762851, -111.31692448971373),zoom=6) # Centered at Big Ski ski area! 

# Add the subregion boundaries as a transparent, black outline
Map.addLayer(ee.Image().paint(CU_river_huc04, 0, 2), {}, 'Columbia Headwaters')
Map.addLayer(ee.Image().paint(CO_river_huc04, 0, 2), {}, 'Colorado Headwaters')
Map.addLayer(ee.Image().paint(MO_river_huc04, 0, 2), {}, 'Missouri Headwaters')

# Add the hydrography data
Map.addLayer(ee.Image().paint(CU_river_streams), visParams, 'Columbia Headwaters Rivers')
Map.addLayer(ee.Image().paint(CO_river_streams), visParams, 'Colorado Headwaters Rivers')
Map.addLayer(ee.Image().paint(MO_river_streams), visParams, 'Missouri Headwaters Rivers')

# Show the map
Map.addLayerControl()
Map

Map(center=[45.2607563762851, -111.31692448971373], controls=(WidgetControl(options=['position', 'transparent_…

## 4. Export the Above Data as KML Files

In [4]:
task = ee.batch.Export.table.toDrive(**{
    'collection': CU_river_huc04,
    'description': 'columbia_headwaters',
    'driveFolder': GDriveSaveFolder, 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': CO_river_huc04,
    'description': 'colorado_headwaters',
    'driveFolder': GDriveSaveFolder, 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': MO_river_huc04,
    'description': 'missouri_headwaters',
    'driveFolder': GDriveSaveFolder, 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': CU_river_streams,
    'description': 'columbia_headwaters_rivers',
    'driveFolder': GDriveSaveFolder, 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': CO_river_streams,
    'description': 'colorado_headwaters_rivers',
    'driveFolder': GDriveSaveFolder, 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': MO_river_streams,
    'description': 'missouri_headwaters_rivers',
    'driveFolder': GDriveSaveFolder, 
    'fileFormat': 'KML',
})
task.start()
task.status()



{'state': 'READY',
 'description': 'missouri_headwaters_rivers',
 'creation_timestamp_ms': 1672372828985,
 'update_timestamp_ms': 1672372828985,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_FEATURES',
 'id': '47GWIXOHRDIZ2NMLV3JNGVUD',
 'name': 'projects/earthengine-legacy/operations/47GWIXOHRDIZ2NMLV3JNGVUD'}

In [6]:
ee.data.getTaskStatus('47GWIXOHRDIZ2NMLV3JNGVUD')

[{'state': 'COMPLETED',
  'description': 'missouri_headwaters_rivers',
  'creation_timestamp_ms': 1672372828985,
  'update_timestamp_ms': 1672372853286,
  'start_timestamp_ms': 1672372833540,
  'task_type': 'EXPORT_FEATURES',
  'destination_uris': ['https://drive.google.com/#folders/1frkhddHlfLsx2IvTW0QQTTmD8fJ-7U6S'],
  'attempt': 1,
  'batch_eecu_usage_seconds': 26.563098907470703,
  'id': '47GWIXOHRDIZ2NMLV3JNGVUD',
  'name': 'projects/earthengine-legacy/operations/47GWIXOHRDIZ2NMLV3JNGVUD'}]

## 5. Get USGS National Elevation Dataset DEMs for Subregions

In [7]:
# Get elevation field from USGS NED data and clip using our subregions created above
CU_huc04_ned = ee.Image('USGS/NED').select('elevation').clip(CU_river_huc04)
CO_huc04_ned = ee.Image('USGS/NED').select('elevation').clip(CO_river_huc04)
MO_huc04_ned = ee.Image('USGS/NED').select('elevation').clip(MO_river_huc04)


## 6. Export Elevation Data

__Note__: This creates a lot of data (as GeoTIFFs) in your Google Drive folder. Only run these when you want to export the data. 

In [8]:
downConfig = {'scale': 10, 'maxPixels': 1.0E13, 'driveFolder': GDriveSaveFolder, 'crs':'EPSG:4326'}  # scale means resolution.

task_up_cda = ee.batch.Export.image(CU_huc04_ned.clip(CU_river_huc04.geometry()), 'CU_huc04_ned', downConfig)
task_up_cda.start()
task_up_cda.status()

task_up_cda = ee.batch.Export.image(CO_huc04_ned.clip(CO_river_huc04.geometry()), 'CO_huc04_ned', downConfig)
task_up_cda.start()
task_up_cda.status()

task_up_cda = ee.batch.Export.image(MO_huc04_ned.clip(MO_river_huc04.geometry()), 'MO_huc04_ned', downConfig)
task_up_cda.start()
task_up_cda.status()


{'state': 'READY',
 'description': 'MO_huc04_ned',
 'creation_timestamp_ms': 1672373147593,
 'update_timestamp_ms': 1672373147593,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'WIAQZJCDXK43NSNVIGXGT6KN',
 'name': 'projects/earthengine-legacy/operations/WIAQZJCDXK43NSNVIGXGT6KN'}

In [11]:
# Check Task Status
ee.data.getTaskStatus('WIAQZJCDXK43NSNVIGXGT6KN')

[{'state': 'COMPLETED',
  'description': 'MO_huc04_ned',
  'creation_timestamp_ms': 1672373147593,
  'update_timestamp_ms': 1672378857057,
  'start_timestamp_ms': 1672373175985,
  'task_type': 'EXPORT_IMAGE',
  'destination_uris': ['https://drive.google.com/#folders/1frkhddHlfLsx2IvTW0QQTTmD8fJ-7U6S'],
  'attempt': 1,
  'batch_eecu_usage_seconds': 406316.28125,
  'id': 'WIAQZJCDXK43NSNVIGXGT6KN',
  'name': 'projects/earthengine-legacy/operations/WIAQZJCDXK43NSNVIGXGT6KN'}]